## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

1980it [01:02, 31.71it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 1980


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 1980/1980 [00:08<00:00, 235.86it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Cheyanne Aguilar,From another space in time,Humanist | Disenfranchised Millennial | Multi-...,2010-04-03 02:51:51,986,2174,16269,False,2020-07-30 20:39:30,With Automation &amp; #COVID19 it has become p...,[COVID19],Twitter for iPhone,False
0,Coronavirus Updates - Alexander Higgins,,An independent citizen journalist that aggrega...,2010-05-02 03:52:20,17432,6805,9892,False,2020-07-30 20:39:29,Iowa reports 34 new cases and 1 new death brin...,None,Alexander Higgins,False
0,chewy’s beard,,,2016-12-06 21:43:42,12,60,122,False,2020-07-30 20:39:29,@realDonaldTrump you know you don’t have the a...,None,Twitter for iPhone,False
0,William Imasuen,"Lagos, Nigeria","William is an engineer, a minister & a speaker...",2012-01-04 10:50:49,66,35,38,False,2020-07-30 20:39:29,Bedtime Meal #COVID19 #Trump #lockdownextensio...,"[COVID19, Trump, lockdownextension, heatwave, ...",Twitter for Android,False
0,Emma,,Ginger and Scottish!,2020-06-16 11:29:51,37,59,230,False,2020-07-30 20:39:28,Should the 2m rule be enforced so we can preve...,None,Twitter for Android,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (42456, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 1980 past tweets: 42456 all tweets: 44436


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (44436, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)